In [9]:
# Install Dependencies and Set-up
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
from google.colab import files
Train_Dataset = files.upload()

In [ ]:
!unzip train.zip

In [ ]:
from google.colab import files
Valid_Dataset = files.upload()

In [ ]:
!unzip valid.zip

In [ ]:
os.chdir("/content")
os.listdir()

In [16]:
# Load datasets and ensure each image is labeled with a class
# Number of classes: 31
from tensorflow.keras.preprocessing.image import ImageDataGenerator

traingen = ImageDataGenerator(rescale=1./255)
validgen = ImageDataGenerator(rescale=1./255)

train_generator = traingen.flow_from_directory(
    '/content/train',
    target_size=(256, 256),
    batch_size=128,
    class_mode='categorical',
    shuffle=True
)

valid_generator = validgen.flow_from_directory(
    '/content/valid',
    target_size=(256, 256),
    batch_size=128,
    shuffle=True,
    class_mode='categorical'
)

Found 34069 images belonging to 31 classes.
Found 8525 images belonging to 31 classes.


In [19]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dense, Dropout


## Loading InceptionV3 model
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(256,256,3))

finetune_afterwards = 290
for layer in base_model.layers[:finetune_afterwards]:
    layer.trainable = False

GAP_2D = GlobalAveragePooling2D()
Dense1 = Dense(800, activation = 'relu')
Dropout1 = Dropout(0.2)
Dense2 = Dense(300, activation = 'relu')
Dropout2 = Dropout(0.2)
prediction = Dense(31, activation ='softmax')


model = models.Sequential([
    base_model,
    GAP_2D,
    Dense1,
    Dropout1,
    Dense2,
    Dropout2,
    prediction
])



In [20]:
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import EarlyStopping
import pickle

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=2,  restore_best_weights=True)

STEP_SIZE_TRAIN = 266
STEP_SIZE_VALID = 66

run = model.fit(train_generator,
                steps_per_epoch=STEP_SIZE_TRAIN,
                validation_data=valid_generator,
                validation_steps=STEP_SIZE_VALID,
                epochs=5
)

pickle.dump(model, open('model.pkl', 'wb'))

Epoch 1/5
266/266 [==============================] - 4455s 17s/step - loss: 0.4623 - accuracy: 0.8565 - val_loss: 0.2050 - val_accuracy: 0.9287
Epoch 2/5
266/266 [==============================] - 4451s 17s/step - loss: 0.1307 - accuracy: 0.9570 - val_loss: 0.1340 - val_accuracy: 0.9562
Epoch 3/5
266/266 [==============================] - 4484s 17s/step - loss: 0.0798 - accuracy: 0.9730 - val_loss: 0.1165 - val_accuracy: 0.9641
Epoch 4/5
266/266 [==============================] - 4432s 17s/step - loss: 0.0580 - accuracy: 0.9803 - val_loss: 0.1145 - val_accuracy: 0.9635
Epoch 5/5
266/266 [==============================] - 4443s 17s/step - loss: 0.0473 - accuracy: 0.9842 - val_loss: 0.1389 - val_accuracy: 0.9607


In [40]:
# Evaluate the accuracy of the model
scores = model.evaluate(valid_generator, steps=STEP_SIZE_VALID)
print(scores)

66/66 [==============================] - 857s 13s/step - loss: 0.1403 - accuracy: 0.9603
[0.14030654728412628, 0.9603456258773804]
